Preliminary analysis in order to check the relevant information of te a/b testing file such as:

1. Duplicates
2. Nulls
3. Number of groups
4. Relation of the samples
5. Variable types
6. Normality test

Having this information there will be selected the corresponding A/B test that maximizes the tatistic relevance


## Imports


In [1]:
# Libraries
import pandas as pd


In [2]:
#dataframe
df_preliminary_raw = pd.read_csv(r"C:\Users\matis\Desktop\DataAnalyticsPractice\ABTestPractice\ABTestPractice1\data\marketing_AB.csv",usecols=lambda col:not col.startswith('Unnamed'))


In [3]:
df_preliminary_raw.sample(5)

,user id,test group,converted,total ads,most ads day,most ads hour
163926,1612323,ad,False,5,Saturday,12
27130,1606539,ad,False,8,Wednesday,22
410579,1322140,ad,False,8,Wednesday,14
267459,1061715,ad,False,49,Tuesday,18
17632,1557293,ad,False,54,Wednesday,9


## Check for duplicates

In [4]:
#The total number of duplicate rows is counted
df_preliminary_raw.duplicated().sum()

np.int64(0)

## Nulls

In [5]:
df_preliminary_raw.isna().sum()

user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64

## Number of groups
We assume that the independent variable is located in the test group column and it is necessary to verify the different values it is taking

In [6]:
df_preliminary_raw["test group"].value_counts()

test group
ad     564577
psa     23524
Name: count, dtype: int64

Two different values as showed in the instructions: If "ad" the person saw the advertisement, if "psa" they only saw the public service announcement

## Relation of the samples
A crucial aspect is to know if the samples have any relationship. To prove that we compare the different user id values to chech if there hare apparitions in both samples.

In [7]:
df_preliminary_raw["user id"].value_counts().sort_values(ascending=False)

user id
1137664    1
1237779    1
1069124    1
1119715    1
1144181    1
          ..
1146893    1
904721     1
1223573    1
1380378    1
1561741    1
Name: count, Length: 588101, dtype: int64

We prove that there is only one apparition for each id

## Variable Types
variables:

test group: treatment variable that helps to identify whether there is a significant difference between the groups under controlled conditions.

converted: target variable of the ab test

total ads: exposure variables or contextual factors that describe the intensity and distribution of the intervention.	

most ads day: exposure variables or contextual factors that describe the intensity and distribution of the intervention.

most ads hour:exposure variables or contextual factors that describe the intensity and distribution of the intervention.

In [8]:
df_preliminary_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 23.0+ MB


From this pre-analysis it can be concluded that the following methodology will be applied:
1. A/B testing analysis between the test group and converted variables to identify if there are significant differences between applying an advertisement or not.
2. Context analysis: if there is a significant difference in the A/B testing, the exposure variables will be analysed to measure the factors that most influence the conversion of users by analysing total ads, mosta ads day and most ads hour.

## Contextual metrics and sample sizes

In [9]:
# group by test group and calculate statistics
table = df_preliminary_raw.groupby('test group').agg(
    total_users=("user id","nunique"),
    total_ads=('total ads', 'sum'),          # Total adverts seen
    converted=('converted', 'sum')           # Total conversions (True = 1)
).reset_index()

# total conversions and conversion ratio
table['total_conversions'] = table['converted']  # Total conversions 
table['conversion_ratio'] = table['converted'] / table['total_users']  # conversion ratio

# Ver la tabla resultante
print(table)



  test group  total_users  total_ads  converted  total_conversions  \
0         ad       564577   14014701      14423              14423   
1        psa        23524     582481        420                420   

   conversion_ratio  
0          0.025547  
1          0.017854  


## Normality test for target variable
There is no normality test needed due to the boolean type of this variable where 1 is converted and 0 is non converted

## Conclussion: Which A/B test model shall be implemented?

1. 2 Groups: ad and psa
2. Independent samples: there aren't multiple values for each user_id
3. Boolean target variable
4. Samples are of sufficient size for testing


The appropriate statistical test for this A/B test is the **Chi-square test of independence**. This test will help determine if there is a statistically significant difference in conversion rates between the "ad" and "psa" groups.